In [64]:
#installing selenium
!pip install selenium

In [65]:
#installing the required drivers and files to launch Chrome browser
!apt-get update 
!apt install chromium-chromedriver


# remove the --headless argument to see the actions in chrome browser live, otherwise code will run in headless mode with all operations in background
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (140 kB/s)
Reading package lists... Done
Reading package lists... Done
Building de

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


In [0]:
#importing the required libraries
from PIL import Image, ImageDraw

Initial Check to see if the website is running

In [66]:
driver.get("http://covid19visual.herokuapp.com/home")
driver.title

'Home'

Get Baseline Images for country maps

In [75]:
#Run this to get baseline images for maps
# baseline images have already been provided in the folder
#driver.get("http://covid19visual.herokuapp.com/home")
driver.back()
paget = driver.title
print("Currently on page ",paget)
driver.find_element_by_css_selector("#root > div > div > div:nth-child(3) > div.homecontainer > div.countries > div:nth-child(5) > a").click()
paget = driver.title
print("Currently on page ",paget)
time.sleep(5)
element = driver.find_element_by_css_selector("#chart")
driver.execute_script("arguments[0].scrollIntoView();", element)
name = paget + ".png"
driver.save_screenshot(name)

Currently on page  India Impact


NoSuchElementException: ignored

Function to calculate pixel level similarity

In [0]:
def process_region( image, x, y, width, height):
        region_total = 0

        # This can be used as the sensitivity factor, the larger it is the less sensitive the comparison
        factor = 100

        for coordinateY in range(y, y+height):
            for coordinateX in range(x, x+width):
                try:
                    pixel = image.getpixel((coordinateX, coordinateY))
                    region_total += sum(pixel)/4
                except:
                    return
      
        return region_total/factor

Code to Verify Country Maps from Homepage

In [51]:
  
   
     i = 1
     driver.get("http://covid19visual.herokuapp.com/home")
     while (i<=5): 

       # This makes the code go to the homepage, click on the country tab, go to the country page and click a snapshot of the map element  
        if i!=1:
         driver.back()
        paget = driver.title
        print("Currently on page ",paget)
        country_link = "#root > div > div > div:nth-child(3) > div.homecontainer > div.countries > div:nth-child(" + str(i) + ") > a"
        driver.find_element_by_css_selector(country_link).click()
        paget = driver.title
        print("Currently on page ",paget)
        time.sleep(5)
        element = driver.find_element_by_css_selector("#chart")
        driver.execute_script("arguments[0].scrollIntoView();", element)
        name = paget + "_test.png"
        driver.save_screenshot(name)
        i = i + 1
        
        base = paget + ".png"
        screenshot_staging = Image.open(base)
        screenshot_production = Image.open(name)
        columns = 60
        rows = 80
        screen_width, screen_height = screenshot_staging.size

        block_width = ((screen_width - 1) // columns) + 1 # this is just a division ceiling
        block_height = ((screen_height - 1) // rows) + 1
        count = 0  

        #this compares each pixel of the snapshot clicked (staging screenshot) with the baseline image (production screenshot)
        for y in range(0, screen_height, block_height+1):
  
            for x in range(0, screen_width, block_width+1):
                region_staging = process_region(screenshot_staging, x, y, block_width, block_height)
                region_production = process_region(screenshot_production, x, y, block_width, block_height)

                if region_staging is not None and region_production is not None and region_production != region_staging:
                    count = count + 1
                    print("Mismatch  in", paget)
                    draw = ImageDraw.Draw(screenshot_staging)
                    draw.rectangle((x, y, x+block_width, y+block_height), outline = "red")
        #results of the screenshot comparison is stored. If there is a mismatch, it is shown in the results image as red boxes
        result = paget + "_result.png"
        screenshot_staging.save(result)
              
        

Currently on page  Home
Currently on page  India
Currently on page  Home
Currently on page  Usa
Currently on page  Home
Currently on page  Germany
Currently on page  Home
Currently on page  Italy
Currently on page  Home
Currently on page  Singapore


Code to get baseline images for Header Bar section

In [70]:
  
i = 2
driver.get("http://covid19visual.herokuapp.com/home")
#driver.back()
paget = driver.title
print("Currently on page ",paget)
while (i<=5):   
        #if i!=1:
         #driver.back()
        country_link = "#root > div > div > nav > div > div.collapse.navbar-collapse > ul > li:nth-child(" + str(i) + ") > a"
        driver.find_element_by_css_selector(country_link).click()
        paget = driver.title
        print("Currently on page ",paget)
        time.sleep(5)
        #element = driver.find_element_by_css_selector("#chart")
        #driver.execute_script("arguments[0].scrollIntoView();", element)
        check = driver.current_url
        if check == 'http://covid19visual.herokuapp.com/events':
           name = "Events_test.png"
        else: 
           name = paget + "_test.png"
        driver.save_screenshot(name)
        i = i + 1
        

#indiat > div > section > div:nth-child(1) > div > div.chart-content > canvas
#indiat > div > section > div:nth-child(1) > div > div.chart-content > canvas
#indiat > div > section > div > div > div.chart-content > canvas

Currently on page  Home
Currently on page  India Timeline
Currently on page  India Demographics
Currently on page  India Impact
Currently on page  India Impact


Code to test the sections in the Header Bar

In [76]:
i = 2
driver.get("http://covid19visual.herokuapp.com/home")
paget = driver.title
print("Currently on page ",paget)
# The code goes to the homepage, clicks on the section option at the top, goes to that section, and clicks a snapshot of that section
while (i<=5):   
        #if i!=1:
         #driver.back()
        country_link = "#root > div > div > nav > div > div.collapse.navbar-collapse > ul > li:nth-child(" + str(i) + ") > a"
        driver.find_element_by_css_selector(country_link).click()
        paget = driver.title
        print("Currently on page ",paget)
        time.sleep(5)
        #element = driver.find_element_by_css_selector("#chart")
        #driver.execute_script("arguments[0].scrollIntoView();", element)
        check = driver.current_url
        if check == 'http://covid19visual.herokuapp.com/events':
           name = "Events_test.png"
           tester = "Events_eval.png"
        else: 
           name = paget + "_test.png"
           tester = paget + "_eval.png"
        driver.save_screenshot(tester)
        i = i + 1

#comparing the pixels of our snapshot with the baseline image for comparison
        screenshot_staging = Image.open(name)
        screenshot_production = Image.open(tester)
        columns = 60
        rows = 80
        screen_width, screen_height = screenshot_staging.size

        block_width = ((screen_width - 1) // columns) + 1 # this is just a division ceiling
        block_height = ((screen_height - 1) // rows) + 1
        count = 0  
        for y in range(0, screen_height, block_height+1):
  
            for x in range(0, screen_width, block_width+1):
                region_staging = process_region(screenshot_staging, x, y, block_width, block_height)
                region_production = process_region(screenshot_production, x, y, block_width, block_height)

                if region_staging is not None and region_production is not None and region_production != region_staging:
                    count = count + 1
                    print("Mismatch  in", name)
                    draw = ImageDraw.Draw(screenshot_staging)
                    draw.rectangle((x, y, x+block_width, y+block_height), outline = "red")
        result = str(i) + "_pixelmatch.png"
        screenshot_staging.save(result)





Currently on page  Home
Currently on page  India Timeline
Currently on page  India Demographics
Currently on page  India Impact
Currently on page  India Impact


In [0]:
driver.quit()

NameError: ignored